In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from PIL import Image
import albumentations as A
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

torch.cuda.empty_cache()

## Pre-requisite Imports

#### Dataloader transforms

In [ ]:
both_transform = A.Compose(
    [A.Resize(width=256, height=256),], additional_targets={"image0": "image"},
)

transform_only_input = A.Compose(
    [
#         A.HorizontalFlip(p=0.0),
        A.ColorJitter(p=0.2),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)

transform_only_mask = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)

#### Dataloader

In [ ]:
class WordDataset(Dataset):
    def __init__(self, root_dir):
        print(os.getcwd())
        # self.root_dir = os.path.normpath(os.path.join(os.getcwd(), root_dir))
        self.root_dir = root_dir
        # self.list_files = os.listdir(self.root_dir)
        self.list_files = [f'/kaggle/input/typed-and-handwritten-hindi-text/ConcatenatedImages/{i}.png' for i in range(20000, 100000)]
        print(f"{len(self.list_files)} files loaded")

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, index):
        img_file = self.list_files[index]
        img_path = os.path.join(self.root_dir, img_file)
        image = np.array(Image.open(img_path))
        # input_image = image[:, :256, :]
        # target_image = image[:, 256:, :]
        input_image = image[:, :256]
        target_image = image[:, 256:]

        augmentations = both_transform(image=input_image, image0=target_image)
        input_image = augmentations["image"]
        target_image = augmentations["image0"]

        input_image = transform_only_input(image=input_image)["image"]
        target_image = transform_only_mask(image=target_image)["image"]

        return input_image, target_image

#### generator

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act='relu', use_dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride=2, padding=1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act=='relu' else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down
    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x

class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        self.down1 = Block(features, features*2, down=True, act='leaky', use_dropout=False)
        self.down2 = Block(features*2, features*4, down=True, act='leaky', use_dropout=False) # 32
        self.down3 = Block(features*4, features*8, down=True, act='leaky', use_dropout=False) # 16
        self.down4 = Block(features*8, features*8, down=True, act='leaky', use_dropout=False) # 8
        self.down5 = Block(features*8, features*8, down=True, act='leaky', use_dropout=False) # 4
        self.down6 = Block(features*8, features*8, down=True, act='leaky', use_dropout=False) # 2

        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1), # 1 x 1
            nn.ReLU(),
        )

        self.up1 = Block(features*8, features*8, down=False, act='relu', use_dropout=True)
        self.up2 = Block(features*8*2, features*8, down=False, act='relu', use_dropout=True)
        self.up3 = Block(features*8*2, features*8, down=False, act='relu', use_dropout=True)
        self.up4 = Block(features*8*2, features*8, down=False, act='relu', use_dropout=False)
        self.up5 = Block(features*8*2, features*4, down=False, act='relu', use_dropout=False)
        self.up6 = Block(features*4*2, features*2, down=False, act='relu', use_dropout=False)
        self.up7 = Block(features*2*2, features, down=False, act='relu', use_dropout=False)

        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)

        bottleneck = self.bottleneck(d7)

        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))

        return self.final_up(torch.cat([up7, d1], 1))

#### Utils

In [ ]:
def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    
    state_dict = checkpoint["state_dict"]
    
    # If the keys are prefixed with "module.", remove it
    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith("module."):
            new_state_dict[k[7:]] = v  # Remove the "module." part
        else:
            new_state_dict[k] = v

    # print(new_state_dict.keys())
    model.load_state_dict(new_state_dict)
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

## Generate Results from validation set

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VAL_DIR = "/kaggle/input/typed-and-handwritten-hindi-text/ConcatenatedImages"
MODEL_DIR = "/kaggle/input/hindi-text2ht-pix2pix-generator/pytorch/epoch-58/1/gen.pth.tar"

LEARNING_RATE = 2e-4

gen = Generator(in_channels=1, features=64).to(DEVICE)
gen.eval()
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
load_checkpoint(os.path.normpath(os.path.join(os.getcwd(), MODEL_DIR)), gen, opt_gen, LEARNING_RATE)

In [ ]:
val_dataset = WordDataset(root_dir=os.path.normpath(os.path.join(os.getcwd(), VAL_DIR)))
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False)

In [10]:
import os
og_dir, gen_dir, tgt_dir = "/kaggle/working/original", "/kaggle/working/generated", "/kaggle/working/target"

os.makedirs(og_dir, exist_ok=True)
os.makedirs(gen_dir, exist_ok=True)
os.makedirs(tgt_dir, exist_ok=True)

In [ ]:
loop = tqdm(val_loader, leave=True)

batch_size = 100

for idx, (x, y) in enumerate(loop):
    x = x.to(DEVICE)
    y = y.to(DEVICE)

    # Train Discriminator
    with torch.amp.autocast('cuda'):
        y_fake = gen(x)
        for i in range(batch_size):
            og_img = x[i].detach().cpu().numpy()[0]
            gen_img = y_fake[i].detach().cpu().numpy()[0]
            tgt_img = y[i].detach().cpu().numpy()[0]

            plt.imsave(f'/kaggle/working/original/{idx*batch_size + i + 1}.png', og_img, cmap='gray')
            plt.imsave(f'/kaggle/working/generated/{idx*batch_size + i + 1}.png', gen_img, cmap='gray')
            plt.imsave(f'/kaggle/working/target/{idx*batch_size + i + 1}.png', tgt_img, cmap='gray')
            
            # plt.figure(figsize=(9, 3))
            # plt.subplot(1, 3, 1)
            # plt.imshow(og_img, cmap='gray')
            # plt.title(f'Input {i}')
            # plt.subplot(1, 3, 2)
            # plt.imshow(gen_img, cmap='gray')
            # plt.title(f'Generated {i}')
            # plt.subplot(1, 3, 3)
            # plt.imshow(tgt_img, cmap='gray')
            # plt.title(f'Label {i}')
            # plt.show()

In [ ]:
torch.cuda.empty_cache()

## Generate HTR Results for all dirs

In [1]:
!pip install editdistance

In [ ]:
plt.imshow("/kaggle/working/generated/1000.png")

In [2]:
import random, math
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import *
# import tf.keras.backend as K
from keras.activations import elu
import cv2, itertools, sys, editdistance, math
from tensorflow.keras.backend import ctc_batch_cost as ctcLoss

seed = 13
random.seed(seed)
np.random.seed(seed)

tf.keras.backend.clear_session()

In [ ]:
import os

_, _, files = next(os.walk("/kaggle/working/original"))
file_count = len(files)
print(file_count)

#### data_utils.py

In [3]:
def truncateLabel(text, maxStringLen = 32):
	cost = 0
	for i in range(len(text)):
		if i!=0 and text[i] == text[i-1]:
			cost+=2
		else:
			cost+=1
		if cost > maxStringLen:
			return text[:i]
	return text

def textToLabels(text, unicodes):
	ret = []
	for c in text:
		ret.append(unicodes.index(c))
	return ret

def labelsToText(labels, unicodes):
	ret = []
	for c in labels:
		if c == len(unicodes):
			ret.append("")
		else:
			ret.append(unicodes[c])
	return "".join(ret)

def preprocess(img, dataAugmentation = False):
	(wt, ht) = (128, 32)
	if img is None:
		img = (np.zeros((wt, ht, 1))).astype('uint8')
	img = cv2.threshold(img, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] * 255

	if dataAugmentation:
		stretch = (random.random() - 0.5) 						# -0.5 .. +0.5
		wStretched = max(int(img.shape[1] * (1 + stretch)), 1)  # random width, but at least 1
		img = cv2.resize(img, (wStretched, img.shape[0])) 		# stretch horizontally by factor 0.5 .. 1.5
	img = closeFit(img)                                         # to avoid lot of white space around text

	h = img.shape[0]
	w = img.shape[1]
	fx = w / wt
	fy = h / ht
	f = max(fx, fy)
	newSize = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1)) 	#scale according to f (result at least 1 and at most wt or ht)
	img = cv2.resize(img, newSize, interpolation = cv2.INTER_AREA)   		#INTER_AREA important, Linear loses all info
	img = cv2.threshold(img, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] * 255

	target = np.ones([ht, wt]) * 255
	target[0:newSize[1], 0:newSize[0]] = img
	img = cv2.transpose(target)
	(m, s) = cv2.meanStdDev(img)
	m = m[0][0]
	s = s[0][0]
	img = img - m
	img = img / s if s>1e-3 else img
	return np.reshape(img, (img.shape[0], img.shape[1], 1))

def closeFit(img):
	i = 2
	col = 255 - np.sum(img, axis=0)/img.shape[0]
	while i<img.shape[1] and col[i]<=5:
		i+=1
	w1 = max(0,i - 15)
	i = img.shape[1]-1
	while i>=0 and col[i]<=5:
		i-=1
	w2 = i + 15

	row = 255 - np.sum(img, axis=1)/img.shape[1]
	i = 2
	while i<img.shape[0] and row[i]<=4:
		i+=1
	h1 = max(0,i - 20)
	i = img.shape[0] - 1
	while i>=0 and row[i]<=5:
		i-=1
	h2 = i + 20
	final = img[h1:h2,w1:w2]
	if final.shape[0]*final.shape[1] == 0:
		return img
	return final

## model_utils.py

In [13]:
def predictImage(imgPath, weightPath):
	img = cv2.imread(imgPath, 0)
	img = preprocess(img, False)
	img = np.reshape(img, (1, img2.shape[0], img2.shape[1], 1))
	unicodes = list(np.load('/kaggle/input/hindi-htr-unicodes-weights/unicodes (1).npy', allow_pickle = True))
	model = CRNN(False, len(unicodes + 1))
	model.load_weights(weightPath)
	out = model.predict(img2)
	pred = decode(out)
	print('Recognized Word: '+ str(pred))

def ctcLambdaFunc(yPred, labels, inputLength, labelLength):
	yPred = yPred[:,2:,:]
	loss = ctcLoss(labels, yPred, inputLength, labelLength)
	return loss

def decode(yPred, unicodes):  #Best Path Decoder
	texts = []
	for y in yPred:
		label = list(np.argmax(y[2:],1))
		label = [k for k, g in itertools.groupby(label)]
		text = labelsToText(label, unicodes)
		texts.append(text)
	return texts

def test(model, loader):
	validation = loader.valSet
	trueText = []
	for (i, path) in validation:
		trueText.append(i)

	# Wrap the output of loader.nextVal in a tuple to match Keras input expectations

	batch_size = 500  # Set this to your desired batch size

	# Define output_signature for the validation data generator
	output_signature = (
		tf.TensorSpec(shape=(batch_size, 128, 32, 1), dtype=tf.float32),
	)

	validation_data = tf.data.Dataset.from_generator(
		lambda: loader.nextVal(batch_size),
		output_signature=output_signature
	)

	outputs = model.predict(validation_data, steps=math.ceil(loader.valLength / batch_size))
	unicodes = list(np.load('/kaggle/input/hindi-htr-unicodes-weights/unicodes (1).npy', allow_pickle = True))
	predText = decode(outputs, unicodes)

	# print(predText)

	wordOK = 0
	wordTot = 0
	charDist = 0
	charTot = 0
	for i in range(len(trueText)):
		#print(predText[i], trueText[i])
		wordOK += 1 if predText[i] == trueText[i] else 0
		wordTot += 1
		dist = editdistance.eval(predText[i], trueText[i])
		charDist += dist
		charTot += len(trueText[i])

	CAR = 100 - 100 * charDist/charTot
	WAR = 100 * wordOK/wordTot
	print('Character Accuracy Rate (CAR):' + str(CAR))
	print('Word Accuracy Rate (WAR):' + str(WAR))
	return (CAR, WAR)

#### CRNN.py

In [5]:
def CRNN(train, outClasses):
	inputShape = (128, 32, 1)
	kernels = [5, 5, 3, 3, 3]
	filters = [32, 64, 128, 128, 256]
	strides = [(2,2), (2,2), (1,2), (1,2), (1,2)]
	rnnUnits = 256
	maxStringLen = 32

	inputs = Input(name = 'inputX', shape = inputShape, dtype = 'float32')
	labels = Input(name='label', shape=[maxStringLen], dtype='float32')
	inputLength = Input(name='inputLen', shape=[1], dtype='int64')
	labelLength = Input(name='labelLen', shape=[1], dtype='int64')

	inner = inputs
	for i in range(len(kernels)):
		inner = Conv2D(filters[i], (kernels[i], kernels[i]), padding = 'same',\
					   name = 'conv' + str(i+1), kernel_initializer = 'glorot_normal') (inner)
		inner = BatchNormalization() (inner)
		inner = Activation(elu) (inner)
		inner = MaxPooling2D(pool_size = strides[i], name = 'max' + str(i+1)) (inner)
	inner = Reshape(target_shape = (maxStringLen,rnnUnits), name = 'reshape')(inner)

	LSF = LSTM(rnnUnits, return_sequences=True, kernel_initializer='glorot_normal', name='LSTM1F') (inner)
	LSB = LSTM(rnnUnits, return_sequences=True, go_backwards = True, kernel_initializer='glorot_normal', name='LSTM1B') (inner)
	LSB = Lambda(lambda inputTensor: tf.keras.backend.reverse(inputTensor, axes=[1]), output_shape=(maxStringLen, rnnUnits)) (LSB)
	LS1 = Average()([LSF, LSB])
	LS1 = BatchNormalization() (LS1)

	LSF = LSTM(rnnUnits, return_sequences=True, kernel_initializer='glorot_normal', name='LSTM2F') (LS1)
	LSB = LSTM(rnnUnits, return_sequences=True, go_backwards = True, kernel_initializer='glorot_normal', name='LSTM2B') (LS1)
	LSB = Lambda(lambda inputTensor: tf.keras.backend.reverse(inputTensor, axes=[1]), output_shape=(maxStringLen, rnnUnits)) (LSB)
	LS2 = Concatenate()([LSF, LSB])
	LS2 = BatchNormalization() (LS2)

	yPred = Dense(outClasses, kernel_initializer='glorot_normal', name='dense2') (LS2)
	yPred = Activation('softmax', name='softmax') (yPred)
	lossOut = Lambda(ctcLambdaFunc, output_shape=(1,), name='ctc') ([yPred, labels, inputLength, labelLength])

	# if train:
	# 	return Model(inputs=[inputs, labels, inputLength, labelLength], outputs=[lossOut, yPred])
	return Model(inputs=[inputs], outputs=yPred)

#### dataloader.py

In [6]:
class DataLoader():
	def __init__(self, trainFile, valFile, unicodes):
		self.unicodes = unicodes
		self.trainFile = trainFile
		self.valFile = valFile
		self.maxStringLen = 32
		self.trainSet = []
		self.valSet = []
		self.trainIndex = 0
		self.valIndex = 0

		if self.trainFile != "":
			self.trainSet = self.importSets(True)
		if self.valFile != "":
			self.valSet = self.importSets(False)
		self.valLength = len(self.valSet)
		self.trainLength = len(self.trainSet)

	def importSets(self, train):
		set = []
		if train:
			file = open(self.trainFile, 'r', encoding='utf-8')
		else:
			file = open(self.valFile, 'r', encoding='utf-8')
		for line in file:
			inUnicodes = True
			if not line or line[0] =='#':
				#Ignoring Erroneous Lines manually skipped with # in file
				continue
			lineSplit = line.strip().split(' ')
			if len(lineSplit) >= 2:
				fileName = lineSplit[0]
				text = truncateLabel(' '.join(lineSplit[1:]))

				for ch in text:
					if not ch in self.unicodes:
						print('Char '+ str(ch)+ ' Not in Unicodes, and Word Omitted')
						#print(ch,('0'+hex(ord(ch))[2:]))
						inUnicodes = False

				if inUnicodes:
					if train:
						set.append((text, fileName))
					else:
						set.append((text, fileName))
			else:
				print(line + 'Check this Line')
		file.close()
		# random.shuffle(set)
		return set

	def nextTrain(self, batchSize):
		while True:
			if self.trainIndex + batchSize >= self.trainLength:
				self.trainIndex = 0
				random.shuffle(self.trainSet)
			ret = self.getBatch(self.trainIndex, batchSize, True)
			self.trainIndex += batchSize
			yield ret

	def nextVal(self, batchSize):
		while True:
			if self.valIndex >= self.valLength:
				self.valIndex = 0
			ret = self.getBatch(self.valIndex, batchSize, False)
			self.valIndex += batchSize
			yield (ret,)

	def getBatch(self, index, batchSize, train):
		if train:
			batch = self.trainSet[index:index + batchSize]
			size = self.trainLength
		else:
			batch = self.valSet[index:index + batchSize]
			size = self.valLength

		imgs = []
		labels = np.ones([batchSize, self.maxStringLen]) * len(self.unicodes)
		inputLength = np.zeros([batchSize, 1])
		labelLength = np.zeros([batchSize, 1])

		for i in range(min(batchSize, size-index)):
			img = cv2.imread(batch[i][1], 0)
			if img is None:
				img = np.zeros((128,32,1))
				print(batch[i][1] + 'is not available')

			img = img.astype('uint8')
			imgs.append(preprocess(img.astype('uint8'), train))
			labels[i, 0:len(batch[i][0])] = textToLabels(batch[i][0], self.unicodes)
			labelLength[i] = len(batch[i][0])
			inputLength[i] = self.maxStringLen - 2

		inputs = {
				'inputX' : np.asarray(imgs),
				'label' : labels,
				'inputLen' : inputLength,
				'labelLen' : labelLength,
					}
		outputs = {'ctc' : np.zeros([batchSize])}
		if train:
			return (inputs, outputs)
		else:
			return imgs

## generate path - label pairs

In [7]:
annotations = None
with open("/kaggle/input/hindi-htr-calam-dataset-annotations/annotations.txt", "r", encoding="utf-8") as f:
    annotations = f.readlines()
    annotations = [x.replace("\n", "") for x in annotations]

In [11]:
start_range, end_range = 20000, 100000
annotation_subset = []
for i in range(start_range - 1, end_range - 1):
    annotation_subset.append(annotations[i])

og_pairs, gen_pairs, tgt_pairs = [], [], []
for i in range(1, end_range - start_range + 1):
    og_pairs.append(f"{og_dir}/{i}.png {annotation_subset[i-1]}\n")
    gen_pairs.append(f"{gen_dir}/{i}.png {annotation_subset[i-1]}\n")
    tgt_pairs.append(f"{tgt_dir}/{i}.png {annotation_subset[i-1]}\n")

with open("/kaggle/working/og_pairs.txt", "w+") as file: file.writelines(og_pairs)
with open("/kaggle/working/gen_pairs.txt", "w+") as file: file.writelines(gen_pairs)
with open("/kaggle/working/tgt_pairs.txt", "w+") as file: file.writelines(tgt_pairs)

In [14]:
unicodes = list(np.load('/kaggle/input/hindi-htr-unicodes-weights/unicodes (1).npy', allow_pickle = True))

og_loader = DataLoader('', "/kaggle/working/og_pairs.txt", unicodes)
gen_loader = DataLoader('', "/kaggle/working/gen_pairs.txt", unicodes)
tgt_loader = DataLoader('', "/kaggle/working/tgt_pairs.txt", unicodes)
testModel = CRNN(False, len(unicodes) + 1)
testModel.load_weights('/kaggle/input/hindi-htr-unicodes-weights/crnn_weights_exp2.h5')

print("\nHTR Metrics on Original Images")
CAR, WAR = test(testModel, og_loader)
print("\nHTR Metrics on Generated Images")
CAR, WAR = test(testModel, gen_loader)
print("\nHTR Metrics on Target Images")
CAR, WAR = test(testModel, tgt_loader)


HTR Metrics on Original Images
160/160 ━━━━━━━━━━━━━━━━━━━━ 147s 907ms/step
Character Accuracy Rate (CAR):82.29901716934125
Word Accuracy Rate (WAR):38.8425

HTR Metrics on Generated Images
160/160 ━━━━━━━━━━━━━━━━━━━━ 146s 913ms/step
Character Accuracy Rate (CAR):23.61925701195733
Word Accuracy Rate (WAR):0.57125

HTR Metrics on Target Images
160/160 ━━━━━━━━━━━━━━━━━━━━ 146s 914ms/step
Character Accuracy Rate (CAR):79.89813444370066
Word Accuracy Rate (WAR):38.37625


In [16]:
tf.keras.backend.clear_session()